In [1]:
from __future__ import print_function, division

from keras.datasets import mnist,fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import os
import numpy as np

class ACGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)
        losses = ['binary_crossentropy', 'sparse_categorical_crossentropy']

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=losses,
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid, target_label = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model([noise, label], [valid, target_label])
        self.combined.compile(loss=losses,
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, 100)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.summary()

        img = Input(shape=self.img_shape)

        # Extract feature representation
        features = model(img)

        # Determine validity and label of the image
        validity = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes, activation="softmax")(features)

        return Model(img, [validity, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = fashion_mnist.load_data()

        # Configure inputs
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0


        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            # idx = np.random.randint(0, X_train.shape[0], batch_size)
            for index in range(nb_batches):
                global_step += 1
                # progress_bar.update(index)

                # get a batch of real images
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]
                # Image labels. 0-9
                img_labels = y_train[index * batch_size:(index + 1) * batch_size]
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # The labels of the digits that the generator tries to create an
                # image representation of
                sampled_labels = np.random.randint(0, 10, (batch_size, 1))

                # Generate a half batch of new images
                gen_imgs = self.generator.predict([noise, sampled_labels])

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(image_batch, [valid, img_labels])
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, sampled_labels])
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator
                g_loss = self.combined.train_on_batch([noise, sampled_labels], [valid, sampled_labels])

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (
                    epoch, global_step, d_loss[0], 100 * d_loss[3], 100 * d_loss[4], g_loss[0]))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    self.sample_images(epoch, global_step)

    def sample_images(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise, sampled_labels])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        if not os.path.isdir('images_acgan_fashion_mnist'):
            os.mkdir('images_acgan_fashion_mnist')
        fig.savefig("images_acgan_fashion_mnist/epoch_%d_step_%d.png" % (epoch,global_step))
        plt.close()


if __name__ == '__main__':
    acgan = ACGAN()
    acgan.train(epochs=40, batch_size=64, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 32)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 3.136720, acc.: 52.34%, op_acc: 12.50%] [G loss: 3.128325]
epoch:0 step:2 [D loss: 3.401370, acc.: 47.66%, op_acc: 10.16%] [G loss: 3.158858]
epoch:0 step:3 [D loss: 3.293901, acc.: 50.00%, op_acc: 13.28%] [G loss: 3.245234]
epoch:0 step:4 [D loss: 3.201884, acc.: 60.16%, op_acc: 11.72%] [G loss: 3.084615]
epoch:0 step:5 [D loss: 3.179311, acc.: 53.91%, op_acc: 14.06%] [G loss: 3.145490]
epoch:0 step:6 [D loss: 3.065003, acc.: 50.78%, op_acc: 19.53%] [G loss: 3.134848]
epoch:0 step:7 [D loss: 3.034504, acc.: 54.69%, op_acc: 13.28%] [G loss: 3.012243]
epoch:0 step:8 [D loss: 3.112973, acc.: 53.12%, op_acc: 14.06%] [G loss: 3.260856]
epoch:0 step:9 [D loss: 2.885216, acc.: 60.94%, op_acc: 20.31%] [G loss: 3.353634]
epoch:0 step:10 [D loss: 3.016426, acc.: 53.91%, op_acc: 17.97%] [G loss: 3.229168]
epoch:0 step:11 [D loss: 2.921501, acc.: 47.66%, op_acc: 18.75%] [G loss: 3.376432]
epoch:0 step:12 [D loss: 2.841245, acc.: 62.50%, op_acc: 19.53%] [G loss: 3.728518]
e

epoch:0 step:100 [D loss: 2.078390, acc.: 72.66%, op_acc: 44.53%] [G loss: 3.360446]
epoch:0 step:101 [D loss: 2.123763, acc.: 68.75%, op_acc: 45.31%] [G loss: 3.265136]
epoch:0 step:102 [D loss: 2.351552, acc.: 58.59%, op_acc: 46.88%] [G loss: 3.176348]
epoch:0 step:103 [D loss: 2.219523, acc.: 59.38%, op_acc: 46.88%] [G loss: 3.219624]
epoch:0 step:104 [D loss: 1.976641, acc.: 74.22%, op_acc: 46.09%] [G loss: 3.359941]
epoch:0 step:105 [D loss: 2.072357, acc.: 75.78%, op_acc: 39.06%] [G loss: 3.189433]
epoch:0 step:106 [D loss: 2.112266, acc.: 61.72%, op_acc: 45.31%] [G loss: 3.174740]
epoch:0 step:107 [D loss: 1.972626, acc.: 74.22%, op_acc: 50.00%] [G loss: 3.519696]
epoch:0 step:108 [D loss: 2.089628, acc.: 75.78%, op_acc: 42.97%] [G loss: 3.287988]
epoch:0 step:109 [D loss: 2.075350, acc.: 60.94%, op_acc: 47.66%] [G loss: 3.006041]
epoch:0 step:110 [D loss: 1.764223, acc.: 82.81%, op_acc: 50.00%] [G loss: 2.949216]
epoch:0 step:111 [D loss: 2.089046, acc.: 57.03%, op_acc: 50.78%]

...

epoch:39 step:37010 [D loss: 0.829373, acc.: 59.38%, op_acc: 92.97%] [G loss: 0.737366]
epoch:39 step:37011 [D loss: 0.773969, acc.: 53.91%, op_acc: 96.09%] [G loss: 0.767553]
epoch:39 step:37012 [D loss: 0.847720, acc.: 57.03%, op_acc: 94.53%] [G loss: 0.757582]
epoch:39 step:37013 [D loss: 0.802424, acc.: 53.91%, op_acc: 95.31%] [G loss: 0.769417]
epoch:39 step:37014 [D loss: 0.903416, acc.: 48.44%, op_acc: 92.97%] [G loss: 0.744298]
epoch:39 step:37015 [D loss: 0.955225, acc.: 53.12%, op_acc: 90.62%] [G loss: 0.696441]
epoch:39 step:37016 [D loss: 0.823194, acc.: 57.03%, op_acc: 95.31%] [G loss: 0.771154]
epoch:39 step:37017 [D loss: 0.790043, acc.: 59.38%, op_acc: 96.09%] [G loss: 0.762360]
epoch:39 step:37018 [D loss: 0.749531, acc.: 52.34%, op_acc: 96.88%] [G loss: 0.760141]
epoch:39 step:37019 [D loss: 0.825665, acc.: 46.88%, op_acc: 94.53%] [G loss: 0.707779]
epoch:39 step:37020 [D loss: 0.847585, acc.: 58.59%, op_acc: 93.75%] [G loss: 0.719965]
epoch:39 step:37021 [D loss: 0.8

epoch:39 step:37108 [D loss: 0.769865, acc.: 50.00%, op_acc: 97.66%] [G loss: 0.749962]
epoch:39 step:37109 [D loss: 0.821662, acc.: 54.69%, op_acc: 92.97%] [G loss: 0.782680]
epoch:39 step:37110 [D loss: 0.793990, acc.: 58.59%, op_acc: 96.88%] [G loss: 0.742969]
epoch:39 step:37111 [D loss: 0.918757, acc.: 53.91%, op_acc: 92.19%] [G loss: 0.756235]
epoch:39 step:37112 [D loss: 0.846302, acc.: 48.44%, op_acc: 93.75%] [G loss: 0.778189]
epoch:39 step:37113 [D loss: 0.851984, acc.: 47.66%, op_acc: 95.31%] [G loss: 0.733539]
epoch:39 step:37114 [D loss: 0.916967, acc.: 57.81%, op_acc: 92.19%] [G loss: 0.749010]
epoch:39 step:37115 [D loss: 0.764406, acc.: 53.12%, op_acc: 96.88%] [G loss: 0.714588]
epoch:39 step:37116 [D loss: 0.876972, acc.: 49.22%, op_acc: 95.31%] [G loss: 0.738703]
epoch:39 step:37117 [D loss: 0.842772, acc.: 52.34%, op_acc: 93.75%] [G loss: 0.748656]
epoch:39 step:37118 [D loss: 0.835867, acc.: 48.44%, op_acc: 95.31%] [G loss: 0.778502]
epoch:39 step:37119 [D loss: 0.7

epoch:39 step:37206 [D loss: 0.824943, acc.: 50.00%, op_acc: 95.31%] [G loss: 0.750698]
epoch:39 step:37207 [D loss: 0.845953, acc.: 44.53%, op_acc: 96.88%] [G loss: 0.709994]
epoch:39 step:37208 [D loss: 0.842871, acc.: 42.97%, op_acc: 94.53%] [G loss: 0.701359]
epoch:39 step:37209 [D loss: 0.866098, acc.: 43.75%, op_acc: 93.75%] [G loss: 0.785620]
epoch:39 step:37210 [D loss: 0.787952, acc.: 53.91%, op_acc: 96.09%] [G loss: 0.733519]
epoch:39 step:37211 [D loss: 0.863291, acc.: 47.66%, op_acc: 95.31%] [G loss: 0.727450]
epoch:39 step:37212 [D loss: 0.885645, acc.: 53.12%, op_acc: 93.75%] [G loss: 0.724316]
epoch:39 step:37213 [D loss: 0.782101, acc.: 53.91%, op_acc: 96.88%] [G loss: 0.783279]
epoch:39 step:37214 [D loss: 0.857665, acc.: 42.97%, op_acc: 96.88%] [G loss: 0.709230]
epoch:39 step:37215 [D loss: 0.860121, acc.: 46.09%, op_acc: 95.31%] [G loss: 0.746102]
epoch:39 step:37216 [D loss: 0.762470, acc.: 50.00%, op_acc: 97.66%] [G loss: 0.784758]
epoch:39 step:37217 [D loss: 0.8

epoch:39 step:37301 [D loss: 0.783758, acc.: 60.16%, op_acc: 95.31%] [G loss: 0.780595]
epoch:39 step:37302 [D loss: 0.846948, acc.: 51.56%, op_acc: 94.53%] [G loss: 0.794720]
epoch:39 step:37303 [D loss: 0.876477, acc.: 46.09%, op_acc: 91.41%] [G loss: 0.767322]
epoch:39 step:37304 [D loss: 0.856611, acc.: 47.66%, op_acc: 93.75%] [G loss: 0.758368]
epoch:39 step:37305 [D loss: 0.816568, acc.: 55.47%, op_acc: 94.53%] [G loss: 0.775045]
epoch:39 step:37306 [D loss: 0.795437, acc.: 56.25%, op_acc: 96.88%] [G loss: 0.758521]
epoch:39 step:37307 [D loss: 0.753267, acc.: 55.47%, op_acc: 97.66%] [G loss: 0.734429]
epoch:39 step:37308 [D loss: 0.844526, acc.: 53.12%, op_acc: 93.75%] [G loss: 0.805690]
epoch:39 step:37309 [D loss: 0.824618, acc.: 49.22%, op_acc: 98.44%] [G loss: 0.768027]
epoch:39 step:37310 [D loss: 0.911702, acc.: 58.59%, op_acc: 90.62%] [G loss: 0.827337]
epoch:39 step:37311 [D loss: 0.756990, acc.: 57.81%, op_acc: 98.44%] [G loss: 0.783472]
epoch:39 step:37312 [D loss: 0.8

epoch:39 step:37395 [D loss: 0.850731, acc.: 58.59%, op_acc: 94.53%] [G loss: 0.725075]
epoch:39 step:37396 [D loss: 0.891850, acc.: 48.44%, op_acc: 91.41%] [G loss: 0.760932]
epoch:39 step:37397 [D loss: 0.849704, acc.: 53.12%, op_acc: 93.75%] [G loss: 0.709949]
epoch:39 step:37398 [D loss: 0.797030, acc.: 52.34%, op_acc: 95.31%] [G loss: 0.725535]
epoch:39 step:37399 [D loss: 0.792321, acc.: 49.22%, op_acc: 97.66%] [G loss: 0.758924]
epoch:39 step:37400 [D loss: 0.801205, acc.: 57.81%, op_acc: 95.31%] [G loss: 0.767128]
epoch:39 step:37401 [D loss: 0.810174, acc.: 56.25%, op_acc: 94.53%] [G loss: 0.730058]
epoch:39 step:37402 [D loss: 0.889371, acc.: 40.62%, op_acc: 94.53%] [G loss: 0.744262]
epoch:39 step:37403 [D loss: 0.933099, acc.: 57.03%, op_acc: 89.84%] [G loss: 0.749290]
epoch:39 step:37404 [D loss: 0.781077, acc.: 53.91%, op_acc: 95.31%] [G loss: 0.786767]
epoch:39 step:37405 [D loss: 0.750383, acc.: 58.59%, op_acc: 96.09%] [G loss: 0.756897]
epoch:39 step:37406 [D loss: 0.8